In [ ]:
import json
from time import time
from pprint import pprint

import requests
import grequests
import cv2
import matplotlib.pyplot as plt
import numpy as np

import handler_testing_utils
reload(handler_testing_utils)

In [ ]:
# URL = 'http://pyml.taxi.tst.yandex.net/remote_quality_control'
# PING_URL = 'http://pyml.taxi.tst.yandex.net/ping'

URL = 'http://pyml-rqc.taxi.dev.yandex.net/remote_quality_control'
PING_URL = 'http://pyml-rqc.taxi.dev.yandex.net/ping'

# URL = 'http://127.0.0.1:5000/remote_quality_control'
# PING_URL = 'http://127.0.0.1:5000/ping'

# URL = 'http://target076i.load.yandex.net/models/remote_quality_control'
# PING_URL = 'http://target076i.load.yandex.net/models/ping'

In [ ]:
# Check whether server is up
!curl $PING_URL -H "Host: pyml.taxi.yandex.net"

### Define function that sends a request to rqc handler

In [ ]:
def construct_check_history(num_checks, comments, dates, results, check_types):
    assert len(comments) == num_checks
    assert len(dates) == num_checks
    assert len(results) == num_checks
    assert len(results) == num_checks
    assert len(check_types) == num_checks
    check_history = []
    for i in range(num_checks):
        check = {u"comment": comments[i],
                 u"pass_datetime": dates[i],
                 u"id": "",
                 u"result": results[i],
                 u"check_type": check_types[i]}
        check_history.append(check)
    return check_history

def send_request(use_ml, car_brand, car_model, car_color, car_lp_number, exam_country, exam_city,
                 num_checks, comments, dates, results, check_types, pass_id, options,
                 front, back, left, right, salon_back, salon_front,
                 url, folder, exam_info_path):
    check_history = construct_check_history(num_checks=num_checks,
                                            comments=comments,
                                            dates=dates,
                                            results=results,
                                            check_types=check_types)
    exam_info = {
        'use_ml': use_ml,
        'exam_id': 'dkk',
        'exam_type': 'dkk',
        'pass_id': pass_id,
        'exam_country': exam_country,
        'exam_city': exam_city,
        'car_brand': car_brand,
        'car_model': car_model,
        'car_year': 2017,
        'car_color': car_color,
        'car_license_plate_number': car_lp_number,
        'check_history': check_history,
        'options': options
    }
    
    exam_info = handler_testing_utils.to_utf(exam_info)
    
    with open(exam_info_path, mode='wb') as f:
        json.dump(exam_info, f, ensure_ascii=False, indent=2)
    
    
    front_value = "'front=@" + folder + '/' + front + ".jpg;type=image/jpeg'"
    back_value = "'back=@" + folder + '/' + back + ".jpg;type=image/jpeg'"
    left_value = "'left=@" + folder + '/' + left + ".jpg;type=image/jpeg'"
    right_value = "'right=@" + folder + '/' + right + ".jpg;type=image/jpeg'"
    salon_front_value = "'salon_front=@" + folder + \
        '/' + salon_front + ".jpg;type=image/jpeg'"
    salon_back_value = "'salon_back=@" + folder + \
        '/' + salon_back + ".jpg;type=image/jpeg'"
    trunk_value = "'trunk=@" + folder + '/' + "trunk.jpg;type=image/jpeg'"
    blank_value = "'blank=@" + folder + '/' + "blank.jpg;type=image/jpeg'"

    start = time()
    file_option = ' -F '
    command = ('curl' + ' ' + '-H "Host: pyml.taxi.yandex.net"'
               + file_option + front_value
               + file_option + back_value
               + file_option + left_value
               + file_option + right_value
               + file_option + salon_front_value
               + file_option + salon_back_value
               + file_option + blank_value
               + file_option + trunk_value
               + file_option + "'exam_info=@" + exam_info_path + ";type=application/json'"
               + ' ' + url)
    resp = !$command
    finish = time()
    time_elapsed = finish - start
    
    try:
        return time_elapsed, command, json.loads(resp[-1])
    except ValueError:
        return time_elapsed, command, resp[-1]

### Test responses

In [ ]:
# id for experiment: 'o'

In [ ]:
# Case 1: correct number, correct brand-model, correct color, no trash, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'human']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_1.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 1.1: correct number, correct brand-model, correct color, no trash, fair block, experiment
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='o', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'human']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_1_1.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 1.2: correct number, correct brand-model, correct color, no trash, fair block,
# unsupported counrty for car number
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Казахстан', exam_city='Москва',
                 pass_id='i', options=['block'],
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'human']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_1_2.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 2: incorrect number, correct brand-model, correct color, no trash, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='Y445KB123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_2.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 3: correct number, incorrect brand-model, correct color, no trash, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Kia', car_model='Rio', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_3.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 4: correct number, correct brand-model, incorrect color, no trash, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Желтый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_4.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 5: correct number, correct brand-model, correct color, trash, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='black', back='black', left='black', right='black',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_5.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 6: incorrect number, correct brand-model, correct color, no trash, unfair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='Y442KB123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'ml']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_6.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 7: incorrect number, correct brand-model, correct color, no trash, fair block, use_ml=False
time_elapded, command, resp =\
    send_request(use_ml=False, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='Y442KB123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_7.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 8: correct number, correct brand-model, correct color, no trash, fair block, no lp number on photos
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='black', back='black', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_8.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 9: correct number, correct brand-model, correct color, no trash, fair block, no lp number on front photo
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'asessor']*3,
                 front='black', back='back', left='left', right='right',
                 salon_back='salon_back', salon_front='salon_front',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_9.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 10: correct number, correct brand-model, correct color, no trash, 
# incorrect interior, interior_exam=True, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Санкт-Петербург',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'human']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='black', salon_front='black',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_10.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# Case 11: correct number, correct brand-model, correct color, no trash, 
# incorrect interior, interior_exam=False, fair block
time_elapded, command, resp =\
    send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                 car_lp_number='M449TK123', exam_country='Россия', exam_city='Екатеринбург',
                 pass_id='i', options=None,
                 num_checks=3,
                 comments=[u'очень грязное тс']*3,
                 dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                 results=[u'block']*3,
                 check_types=[u'human']*3,
                 front='front', back='back', left='left', right='right',
                 salon_back='black', salon_front='black',
                 url=URL, folder='./example_photos/',
                 exam_info_path='./exam_info_files/exam_info_case_11.json')
print 'Time elapsed:', time_elapded, '\n'
pprint(resp)
print command + ' -v | python -m json.tool'

In [ ]:
# CASE 0: test of models/remote_quality_control handler
exam_info = {
    'right': '',
    'back': '',
    'front': '',
    'left': '',
    'salon_back': '',
    'salon_front': '',
    'recognized_license_plates': {"back": "M449TK123", "front": "M449TK123"},
    'dirt_score': 0.1,
    'damage_score': 0.1,
    'seats_score': 0.1,
    'rugs_score': 0.1,
    'seatcase_score': 0.1,
    'interior_trash_score': 0.1,
    'predicted_probabilities_trash': 
    {"front": {"missType": 0.9920274615287781, "trash": 4.6169848499921784e-10, "cropped": 0.9870995879173279, "blurry": 0.004888891242444515},
     "right": {"missType": 0.46031615138053894, "trash": 1.5586203118075903e-13, "cropped": 0.9941107630729675, "blurry": 0.011293338611721992},
     "back": {"missType": 0.9712460041046143, "trash": 2.721851046771917e-07, "cropped": 0.9869416356086731, "blurry": 0.004127767402678728},
     "left": {"missType": 0.3868817687034607, "trash": 1.3242436661120571e-12, "cropped": 0.9586029648780823, "blurry": 0.07527575641870499}},
    'brand_color_recognition': {'color': {'class': u'белый', 'confidence': 1.0}, 
                                'brand': {'class': u'volkswagen jetta', 'confidence': 1.0}},
    'branding_score': 0.1,
    'meta_scores': {u'meta_score_exterior': 0.2785622473647427,
                    u'meta_score_full_exam': 0.3204572585896411,
                    u'meta_score_interior': 0.024516476569602062},
    "use_ml": True,
    "pass_id": 'test_rqc_id',
    "exam_id": "test_rqc_id",
    "options": ['self_employed', 'self_reg'],
    "exam_country": "Россия",
    "exam_city": "Москва",
    "car_brand": "Volkswagen",
    "car_model": "Jetta",
    "car_year": 2017,
    "car_color": "Белый",
    "car_license_plate_number": "M449TK123",
    "check_history": [{"id": "b0bd5efac9994a3f84fa92be8ba67c95",
                      "pass_datetime": "2018-08-15T09:52:11.415811Z",
                      "comment": "госномер указан неверно. Обновить данные в вашем профиле может таксопарк",
                      "result": "block",
                      'check_type': 'human'}, 
                     {"id": "c37d7a5bfa5b42e49e23c16fc967a4e5",
                      "pass_datetime": "2018-08-15T09:50:55.167526Z",
                      "result": "success",
                      "check_type": "human"},
                      {"id": "392405c237df47ea8fc9afc130610c31", 
                       "pass_datetime": "2018-08-15T08:52:12.203816Z",
                       "result": "success",
                       "check_type": "human"},
                      {"id": "0fa612243a694a51b099f1a240e119e9",
                       "pass_datetime": "2018-08-13T13:20:16.660407Z",
                       "result": "success",
                       "check_type": "human"},
                      {"id": "691702d826834447b5027f75bc652514",
                       "pass_datetime": "2018-08-07T11:57:25.230205Z",
                       "result": "success",
                       "check_type": "human"},
                      {"id": "6a4a59f0a4e349d192335b2f821a8ee7",
                       "pass_datetime": "2018-08-01T09:48:45.895766Z",
                       "result": "success",
                       "check_type": "human"},
                      {"id": "5b0dc3e067ce4c6c9cb47821d80f0989",
                       "pass_datetime": "2018-07-27T05:57:16.985684Z",
                       "result": "success",
                       "check_type": "human"}]}

exam_info = handler_testing_utils.to_utf(exam_info)
    
with open('./exam_info_files/exam_info_case_0.json', mode='wb') as f:
    json.dump(exam_info, f, ensure_ascii=False, indent=2)

start = time()
resp = !curl -F 'exam_info=@./exam_info_files/exam_info_case_0.json;type=application/json' -H "Host: pyml.taxi.yandex.net" http://pyml-rqc.taxi.dev.yandex.net/models/remote_quality_control
# resp = !curl -F 'exam_info=@./exam_info_files/exam_info_case_0.json;type=application/json' -H "Host: pyml.taxi.yandex.net" http://127.0.0.1:5000/models/remote_quality_control
finish = time()
print 'Time elapsed:', finish - start, '\n'
try:
    pprint(json.loads(resp[-1]))
except:
    print resp[-1]

### Check response times with threads and without them

In [ ]:
# Case 1: correct number, correct brand-model, correct color, no trash, fair block
for i in range(20):
    time_elapsed, command, resp =\
        send_request(use_ml=True, car_brand='Volkswagen', car_model='Jetta', car_color='Белый', 
                     car_lp_number='M449TK123', exam_country='Россия', exam_city='Москва',
                     pass_id='o', options=['self_employed', 'self_reg', 'newbie'],
                     num_checks=3,
                     comments=[u'очень грязное тс']*3,
                     dates=[u"2018-04-1" + unicode(i) + "T16:27:18" for i in range(0, 3)],
                     results=[u'block']*3,
                     check_types=[u'human']*3,
                     front='front', back='back', left='left', right='right',
                     salon_back='salon_back', salon_front='salon_front',
                     url=URL, folder='./example_photos/',
                     exam_info_path='./exam_info_files/exam_info_case_1.json')
    print time_elapsed

### Make ammo

In [ ]:
ammo_str = handler_testing_utils.make_ammo(
    folder_images='./example_photos',
    exam_info_path='./exam_info_files/exam_info_case_1.json'
)

In [ ]:
with open('ammo_trial.txt', 'wb') as f:
    f.write(ammo_str.lstrip('\n'))

### Requests checks

### Trivial Checks

In [ ]:
data = {}
for view in ['front', 'back', 'left', 'right']:
    with open('./example_photos/' + view + '.jpg', 'rb') as f:
        img_str = f.read()
    data[view] = img_str

PHOTO_TO_NUMBER_MAPPING = {
    'back': '0',
    'front': '1',
    'left': '2',
    'right': '3'
}
session_trash = grequests.Session()
session_trash.mount('http://', requests.adapters.HTTPAdapter(
    max_retries=requests.packages.urllib3.util.Retry(
        total=(3),
        backoff_factor=0,
        method_whitelist=frozenset(['POST']),
        status_forcelist=[500],
        raise_on_redirect=True,
        raise_on_status=True
    )
))

session_lp = grequests.Session()
session_trash.mount('http://', requests.adapters.HTTPAdapter(
    max_retries=requests.packages.urllib3.util.Retry(
        total=(3),
        backoff_factor=0,
        method_whitelist=frozenset(['POST']),
        status_forcelist=[500],
        raise_on_redirect=True,
        raise_on_status=True
    )
))

In [ ]:
resps = handler_testing_utils.send_parallel_requests_mlaas(data, session_trash)
resps = handler_testing_utils.send_parallel_requests_cv(data, session_lp)

### parse_driver_license checks

In [ ]:
from copy import deepcopy

import requests
import grequests

In [ ]:
# url_pymlaas_dl = 'http://127.0.0.1:5000/parse_driver_license'
url_pymlaas_dl = 'http://pyml.taxi.dev.yandex.net/parse_driver_license'

In [ ]:
prod = '8b3818d4-259d-46e6-89be-cd266f5ae81f'
test = '49f1d123-e803-4fda-b727-62ed0e3c7a4a'

In [ ]:
resp = !curl $url_pymlaas_dl \
        -F "front_binary=@./example_photos/driver_license_ru.jpg" \
        -F "country=rus" \
        -H "Host: pyml.taxi.yandex.net" -v
try:
    pprint(json.loads(resp[-1]))
except:
    pprint(resp[-1])

In [ ]:
print 'first_name:', json.loads(resp[-1])['first_name']['value']
print 'last_name:', json.loads(resp[-1])['last_name']['value']
print 'middle_name:', json.loads(resp[-1])['middle_name']['value']
print 'number:', json.loads(resp[-1])['number']['value']
print 'series:', json.loads(resp[-1])['series']['value']
print 'due_date:', json.loads(resp[-1])['due_date']['value']
print 'issue_date:', json.loads(resp[-1])['issue_date']['value']
print 'birth_date:', json.loads(resp[-1])['birth_date']['value']

In [ ]:
for i in range(1):
    resp = !curl $url_pymlaas_dl \
            -F "front_binary=@./example_photos/blank.jpg" \
            -F "country=rus" \
            -H "Host: pyml.taxi.yandex.net" -v
    try:
        pprint(json.loads(resp[-1]))
    except:
        pprint(resp[-1])

In [ ]:
files = {}
with open('./example_photos/blank.jpg', 'rb') as f:
    photo_binary = f.read()

files['file'] = photo_binary
files['langName'] = 'eng,rus'
files['apikey'] = test
files['meta'] = json.dumps({"StrategyName": "FullOcrVU"})
files_no_api = deepcopy(files)

req_success = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files,
    timeout=3
)

req_timeout = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files,
    timeout=0.1
)

files_no_api.pop('apikey')
req_no_api = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files_no_api,
    timeout=3
)

glist = grequests.map([req_timeout]*2 + [req_no_api]*2 + [req_success]*2,
                      exception_handler=exception_handler)
# glist = grequests.map([req_success]*2, exception_handler=exception_handler)

In [ ]:
glist

In [ ]:
!curl -kD - "https://api-translate.ocr.yandex.net:443/recognize" \
-F "file=@./example_photos/blank.jpg" \
-F "apikey=49f1d123-e803-4fda-b727-62ed0e3c7a4a" \
-F "langName=eng,rus" \
-H "Expect:" \
-F "meta={\"StrategyName\":\"FullOcrVU\"}"

### Trivia